<a href="https://colab.research.google.com/github/flawnson/scripts_n_snippits/blob/master/Colab_mania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Desperate times, desperate measures 😛

# Existing Support

As it currently stands (May 11th, 2020), Google Colab provides integration with Github for .ipynb files, allowing you to search for notebooks that you are affiliated with (Private organizations not included). However, the notebooks you receive from Github are meant to be run as standalone projects, and does not allow you to import custom modules from within the project. [This link](https://colab.research.google.com/github/) will take you to the portal.

According to [the FAQ](https://research.google.com/colaboratory/faq.html), Google Colab allows for up to 12 hours of GPU and TPU compute at a given session, before automatically disconnecting you. Provided GPUs include Nvidia K80s, T4s, P4s and P100s. The amount of available memory changes in accordance to the provided VM. Another limitation is the idle period; Google Colab will end sessions and disconnect from runtime if it detects no activity for a variable amount of time (90 minutes according to [this Stackexchange post](https://stackoverflow.com/questions/54057011/google-colab-session-timeout)). The hard limits are therefore 12 hours of time, and x amount of memory, while the soft limits are an undefined idle time.

The goal therefore, is 3-fold:
*   Extend Colab usage to any python file (.ipynb and .py)
*   Allow the integration of any Github repository, public or private
*   Prevent automatic session termination after the idle limit is exceeded



# Running local projects

Colab allows the use of shell common commands, which can be uesd to access the VM directly instead of through the user interface. Read [this](https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.05-IPython-And-Shell-Commands.ipynb) for more info. To understand magics in IPython, have a look at [this Jupyter Notebook](https://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb). Note that you have root access to the notebook VM. You can list the all files and directories in your current directory with:
```
%ls -al
```
You can also veiw a file on the VM with
```
%less ~/something.txt
```
The file will appear on the right side as a tab that you can save (Ctrl-S) and close as needed

Click the folder icon on the left sidebar and upload your project directory into Colab. You will have to zip the project. You can unzip upon successfully uploading the project with:

In [ ]:
!unzip my_project.zip

Optionally, you may mount Google Drive and access project files and directories through it instead. **Note that regardless of how you upload files to Colab, terminating a session or ending runtime will delete all files and revert mounted Drives.** Once the project is on Colab, change directory into your project file, and run your entry point file as you would from a terminal.


In [ ]:
%cd sample_data

!python main.py

If you have libraries needed to run your code, check that Colab doesn't have them preinstalled already with 
```
!pip freeze
```
If you can't find your library in the list either `!pip install some_package`, or create a `requirements.txt` file with the same format as the pip freeze ouput and run:
```
!pip install -r requirements.txt
```
Note that many python packages available on Windows are not available on Linux, hence you can also install libraries manually, for example:

In [ ]:
%pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
%pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
%pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
%pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
%pip install torch-geometric

# Running Github repositories

Treat Colab like as you would a linux machine from shell.

### Running public repositories

Super straightforward. Change directories with `%cd` on the VM to whichever you fancy. It may be be desireable to change into the `drive` directory after mounting Drive, otherwise your project will be deleted if the session dies or runtime ends. Then in your directory of choice:

In [ ]:
!git clone git@github.com:username/Repo_name.git

Now you can treat the directory the same had you uploaded a project from local.

### Running private repositories

Start by setting global configurations for github on the VM

In [ ]:
!git config --global user.email "flawnsontong1@gmail.com"
!git config --global user.name "flawnson"

Check that you have set git user properly with:

In [ ]:
!git config --list

Check that there is a .ssh directory in root and create one if it doesn't exist. Then create a new key with id_rsa as it's name (it must be id_rsa otherwise Git won't know where to look).

In [ ]:
!ssh-keygen -f ~/.ssh/id_rsa -t rsa

Use cat (will output the file directly) or less (will open a little window on the right with the file) and copy the entire contents of the file. 

In [ ]:
!cat ~/.ssh/id_rsa.pub

Open the id_rsa.pub file (your public key) and copy to clipboard. Paste the copied contents on Github by going to `Github profile -> Settings -> SSH and GPG Keys -> Add new SSH key`. Then you must add Github to the list of authorized hosts.

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

Finally you can git clone your private repository from the ssh link:

In [ ]:
!git clone git@github.com:username/project_name.git

Change directories into your project and run your project via entry point file:

In [ ]:
!python main.py -c /content/DruGgable_Reloaded/config/quinary_config.json -b false

# Stopping Idle Termination

Desperate times desperate measures.


### SSH from local for htop

There is a python package someone created meant to enable ssh access to a Colab notebooks. So far it has worked as expected. After running the code below and launching ssh from WSL, htop seems to successfully prevent idle-termination. Sourced from this [stack overflow post](https://stackoverflow.com/a/61682149).

In [ ]:
# Install colab_ssh
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh
launch_ssh('YOUR_NGROK_AUTH_TOKEN', 'SOME_PASSWORD')

Where YOUR_NGROK_AUTH_TOKEN is your auth token after signing up for ngrok and 'SOME_PASSWORD' is a password or your choice


Use `ssh -p PORT_NUM root@0.tcp.ngrok.io` (example command) in your local terminal to ssh into the notebook's VM.

Where PORT_NUM is the port number that will be provided upon executing launch_ssh

Thie mthod is a simplification of strategies used by sources such as [this](https://medium.com/@meet_patel/how-to-ssh-into-google-colab-and-run-scripts-from-terminal-instead-of-jupyter-notebook-3931f2674258) and [this](https://medium.com/@archie9211/guide-to-connect-to-google-colab-with-ssh-from-terminal-and-run-jupyter-lab-7ed60258368), which require installation of ngrok. As of May 12th, I have successfully kept a VM alive (Nvidia Tesla P100 for PCIe-Based Servers) for 5.5 hours

Attempt #2, kept a VM alive (Nvidia Tesla K80) for 

# ---***WARNING***---
If your session is terminated either automatically or by going to `Runtime` and changing or ending the session, you will have to restart from the beginning (treat it as a new session)

# ---***Further Tests***---
[ ] - Test if cloned repos on mounted Drive remain tracked (probably not)

[ ] - [Test if running macros that create bogus on-screen effects prevent idle termination ](https://stackoverflow.com/a/57114793)

[x]: PASSED - Test if .vimrc is detected when cloned from repo into root dir

[x]: FAILED - [Test if modifying ssh config helps prevent idle termination](https://unix.stackexchange.com/questions/150402/what-is-the-default-idle-timeout-for-openssh)

[x]: PASSED - Test if you can ssh into the notebook VM directly

[x]: FAILED - Test if you can run tmux to keep session awake (may be able to keep sessions running in a tmux server even when the runtime is terminated)

# Executable Script
The following script has been tested successfully for sshing into Colab's VM. After executing the script you should be able to clone into any public or private repos you have been given permission to access. Whether or not Colab's automatic session termination after the idle limit is exceeded still persists has been untested, so for now you probably have to baby the VM and Colab session upon sshing in.

In [1]:
%%bash

# Setup git's credentials on system (verify with git config --list)
git config --global user.email "flawnsontong1@gmail.com"
git config --global user.name "flawnson"

EMAIL='flawnsontong1@gmail.com'
USERNAME='flawnson'
PASSWORD='Orphic7Eunoia3'

# Add keep-alive signals to system as root user (verify with whoami)
echo "ClientAliveInterval 60" >> /etc/ssh/ssh_config

# Only needed if you add ^^^ after sshing into the session (not tested)
# Setup ssh key for connection
echo y | echo None | echo None | ssh-keygen -f ~/.ssh/id_rsa -t rsa | echo y # Redundant echo y in case there is an existing public key
ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# Installing Github's REST API (JS version) to send ssh key to programmatically add ssh key to github.com user account 
npm install @octokit/rest

curl -u "${USERNAME}:${PASSWORD}" \
    --data "{\"title\":\"ssh_colab_`date +%Y%m%d%H%M%S`\",\"key\":\"`cat ~/.ssh/id_rsa.pub`\"}" \
    https://api.github.com/user/keys

# Install colab_ssh tool (uses ngrok in the backend)
sudo apt-get install update
pip install colab_ssh --upgrade

y
+ @octokit/rest@18.0.0
added 40 packages from 64 contributors in 11.23s
{
  "id": 44154717,
  "key": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC+6v55CPoyuiW9QBnx3a0uLL0qAhWhx0ghcOmtRlxkrkH1z9c17WjoVyvwWjEfoBcsiez7N8R2nXgm+QzKifZxcVOJc2TEvMoFA9RJ821tWdfICEEZCyZPeIt+jhNZDtubEKPdKR1xHVNAEsdINK6DzXlMWAl6AJ0DbzQiaDR5n/0TAGwTcVvyl2CJ+79lQJXQ+hsnNAulrrlpJeK8VWKSs16mgirPQjyqC86PmhVRU4AuGXHgr/dbE1ugUX3wEdx5V2EZ7SJQe18ycsl8LWkgt9Wv3ooZo3QQ8DIZY+57UDRGiOVADNdOCli6Rb5H7ABpssr1RG+4jb86RXdn80UR",
  "url": "https://api.github.com/user/keys/44154717",
  "title": "ssh_colab_20200620231728",
  "verified": true,
  "created_at": "2020-06-20T23:17:29Z",
  "read_only": false
}
Reading package lists...
Building dependency tree...
Reading state information...


Enter passphrase (empty for no passphrase): Enter same passphrase again: Enter passphrase (empty for no passphrase): Enter same passphrase again: # github.com:22 SSH-2.0-babeld-7c96ae41
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1032  100   589  100   443    667    502 --:--:-- --:--:-- --:--:--  1168
E: Unable to locate package update


In [ ]:
# If all of the above executes properly, execute the following YOUR_NGROK_AUTH_TOKEN
!pip3 install colab-ssh
!python3 -c "from colab_ssh import launch_ssh; launch_ssh('YOUR_NGROK_AUTH_TOKEN', 'SOME_PASSWORD')"

# Where YOUR_NGROK_AUTH_TOKEN is your auth token after signing up for ngrok and 'SOME_PASSWORD'
# is a password or your choice

# Open your favorite shell and execute the following:
# ssh -p PORT_NUM root@0.tcp.ngrok.io

# Where PORT_NUM is the ssh port number
# default port number for ssh is 22, but use the one provided after executing the above command